# Test of veracity prediction

In [1]:
from dotenv import load_dotenv
from pathlib import Path
import os
import pytorch_lightning as pl

dotenv_path = Path('aic_averitec/.env')
load_dotenv(dotenv_path)

DATASTORE_PATH = os.environ.get("DATASTORE_PATH")
DATASET_PATH = os.environ.get("DATASET_PATH")
MODELS_PATH = os.environ.get("MODELS_PATH")

%load_ext autoreload
%autoreload 2

DEV_PATH = str(os.path.join(DATASET_PATH, 'dev.json'))

In [2]:
import json
import tqdm
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
from aic_averitec.src.models.SequenceClassificationModule import SequenceClassificationModule
from aic_averitec.src.prediction.veracity_prediction import SequenceClassificationDataLoader

LABEL = [
    "Supported",
    "Refuted",
    "Not Enough Evidence",
    "Conflicting Evidence/Cherrypicking",
]

LABEL_TO_IDX = {label: idx for idx, label in enumerate(LABEL)}

## Load evidence files

In [3]:
#load examples
CLAIM_WITH_EVIDENCE_FILE_ORIG = str(os.path.join(DATASTORE_PATH, 'dev_top_3_rerank_qa.json'))
CLAIM_WITH_EVIDENCE_FILE_DEV500 = "aic_averitec/data_store/dev500_questions.jsonl"

examples_orig = []
with open(CLAIM_WITH_EVIDENCE_FILE_ORIG) as f:
    for line in f:
        examples_orig.append(json.loads(line))

print(examples_orig)

examples_dev500 = []
with open(CLAIM_WITH_EVIDENCE_FILE_DEV500) as f:
    for line in f:
        examples_dev500.append(json.loads(line))

print(examples_dev500)

[{'claim_id': 0, 'claim': 'In a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.', 'evidence': [{'question': 'Did Sean connery send a fake letter about real Steve jobs?', 'answer': 'Also, fake Sean Connery sent a letter to Real Steve Jobs.', 'url': 'https://www.nbcnews.com/news/world/pre-caffeine-tech-apple-gossip-smart-pugs-flna122578'}, {'question': 'Did President Trump tell the truth about the fake letter to Apple CEO Steve Job Jobs Jobs  Question answer:  Yes, Trump said it was fake.?', 'answer': 'Thanks to the confluence of my interests and the fact that it’s funny as hell, I’ve been inundated with email regarding Scoopertino’s fake 1998 letter from Sean Connery to Steve Jobs.', 'url': 'https://maclalala2.wordpress.com/2011/06/24/%E3%81%9F%E3%81%8B%E3%81%8C%E3%82%B3%E3%83%B3%E3%83%94%E3%83%A5%E3%83%BC%E3%82%BF%E3%82%BB%E3%83%BC%E3%83%AB%E3%82%B9%E3%83%9E%E3%83%B3%E3%81%AE%E3%81%9F%E3%82%81%E3%81%AB%E3%82%B8%E3%82%A7%E3%83%BC/'}, {'question': 'Did Steve 

In [6]:
#load examples from dev.json (ONLY IF YOU WANT TO USE THE ORIGINAL DEV FILE)
with open(DEV_PATH) as f:
    dev_data = json.load(f)

examples_gold = []
for data_point in dev_data:

    evidence = []
    for question in data_point["questions"]:
        q = question["question"]
        for answer in question["answers"]:
            a = answer["answer"]
            if answer["answer_type"] == "Boolean":
                evidence.append({"question": q, "answer": a + " " + answer["boolean_explanation"]})
            else:
                evidence.append({"question": q, "answer": a})
    ex = {
        "claim": data_point['claim'],
        "evidence": evidence,
    }

    examples_gold.append(ex)

print(examples_gold)


[{'claim': 'In a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.', 'evidence': [{'question': 'Where was the claim first published', 'answer': 'It was first published on Sccopertino'}, {'question': 'What kind of website is Scoopertino', 'answer': 'Scoopertino is an imaginary news organization devoted to ferreting out the most relevant stories in the world of Apple, whether or not they actually occurred - says their about page'}]}, {'claim': 'Trump Administration claimed songwriter Billie Eilish Is Destroying Our Country In Leaked Documents', 'evidence': [{'question': 'Has the Trump administration voiced that Billie Eilish was destroying the country', 'answer': 'A Washington Post story wrongly claimed the Trump administration accused Billie Eilish of “destroying our country'}]}, {'claim': "Due to Imran Khan's criticism of Macron's comments on Islam, French authorities cancelled the visas of 183 Pakistani citizens and deported 118 from the country.", 'evidence

In [7]:
#get y_true
dev_data = json.load(open(DEV_PATH))

dev_labels = [example["label"] for example in dev_data]
y_true = [LABEL_TO_IDX[label] for label in dev_labels]
print(y_true)

print(np.unique(y_true, return_counts=True))

[1, 1, 1, 1, 1, 1, 0, 0, 1, 2, 3, 3, 1, 0, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 0, 0, 3, 0, 1, 0, 1, 1, 1, 1, 1, 0, 3, 1, 1, 1, 3, 1, 1, 3, 1, 1, 0, 1, 1, 1, 0, 3, 2, 3, 1, 1, 0, 1, 1, 0, 1, 2, 1, 1, 1, 1, 3, 1, 0, 2, 0, 1, 1, 1, 3, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 3, 3, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 3, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 2, 0, 1, 0, 1, 0, 1, 2, 3, 0, 0, 0, 0, 3, 1, 1, 1, 0, 2, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 2, 0, 1, 2, 1, 1, 1, 1, 1, 2, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 1, 1, 0, 2, 1, 0, 1, 2, 1, 0, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 3, 1, 1, 0, 1, 1, 1, 1, 0, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 0, 1, 3, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 3, 1, 0, 3, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 3, 1, 1, 0, 2, 1, 0, 1, 1, 

## Original BERT

In [17]:
#needed older transformers version
%pip install transformers==4.29.2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached transformers-4.29.2-py3-none-any.whl.metadata (112 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  You can safely remove it manually.
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.3
    Uninstalling transformers-4.42.3:
      Successfully uninstalled transformers-4.42.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyreft 0.0.5 requires numpy>=1.26.4, but you have numpy 1.22.3 which is incompatible.
pyreft 0.0.5 r

In [17]:
bert_model_name = "bert-base-uncased"
best_checkpoint = str(os.path.join(MODELS_PATH, 'bert_veracity.ckpt'))

tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = BertForSequenceClassification.from_pretrained(
    bert_model_name, num_labels=4, problem_type="single_label_classification"
)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
trained_model = SequenceClassificationModule.load_from_checkpoint(
    best_checkpoint, tokenizer=tokenizer, model=bert_model
).to(device)

dataLoader = SequenceClassificationDataLoader(
    tokenizer=tokenizer,
    data_file="this_is_discontinued",
    batch_size=32,
    add_extra_nee=False,)

/home/mlynatom/venvs/py3.10.4/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassific

In [22]:
predictions = []

example_supports = []

y_pred_ternary = []
y_pred_us = []

for example in tqdm.tqdm(examples):
    example_strings = []
    for evidence in example["evidence"]:
        example_strings.append(
            dataLoader.quadruple_to_string(
                example["claim"], evidence["question"], evidence["answer"], ""
            )
        )

    if (
        len(example_strings) == 0
    ):  # If we found no evidence e.g. because google returned 0 pages, just output NEI.
        example["label"] = "Not Enough Evidence"
        continue

    tokenized_strings, attention_mask = dataLoader.tokenize_strings(example_strings)
    logits = trained_model(
            tokenized_strings.to(device), attention_mask=attention_mask.to(device)
        ).logits
    
    example_support = torch.argmax(
        logits,
        axis=1,
    )

    example_supports.append(example_support)


    has_unanswerable = False
    has_true = False
    has_false = False

    for v in example_support:
        if v == 0:
            has_true = True
        if v == 1:
            has_false = True
        if v in (
            2,
            3,
        ):  # TODO another hack -- we cant have different labels for train and test so we do this
            has_unanswerable = True

    if has_unanswerable:
        answer = 2
    elif has_true and not has_false:
        answer = 0
    elif not has_true and has_false:
        answer = 1
    else:
        answer = 3

    y_pred_ternary.append(answer)

    if has_true and has_false:
        answer_us = 3
    elif has_true and not has_false:
        answer_us = 0
    elif not has_true and has_false:
        answer_us = 1
    else:
        answer_us = 2 #otherwise NEI


    y_pred_us.append(answer_us)

    

#stack the supports to a tensor
example_supports = torch.stack(example_supports)



100%|██████████| 500/500 [00:07<00:00, 65.47it/s]


RuntimeError: stack expects each tensor to be equal size, but got [2] at entry 0 and [1] at entry 1

In [19]:
print(np.unique(example_supports.detach().cpu().numpy(), return_counts=True))

(array([0, 1, 2]), array([ 418, 1070,   12]))


In [12]:
example_supports

[tensor([1, 1, 0, 1, 1, 1, 0, 1, 1], device='cuda:0'),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'),
 tensor([0, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'),
 tensor([1, 1, 1, 1, 0, 1, 1, 1, 1], device='cuda:0'),
 tensor([0, 1, 1, 1, 0, 0, 0, 0, 0], device='cuda:0'),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'),
 tensor([0, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'),
 tensor([0, 0, 0, 0, 0, 1, 1, 0, 0], device='cuda:0'),
 tensor([0, 0, 1, 1, 0, 0, 0, 1, 0], device='cuda:0'),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'),
 tensor([0, 0, 1, 1, 0, 1, 1, 1, 1], device='cuda:0'),
 tensor([1, 1, 1, 0, 1, 1, 1, 1, 1], device='cuda:0'),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0'),
 tensor([1, 1, 0, 1, 0, 0, 0, 0, 0], device='cuda:0'),
 tensor([0

In [13]:
print(np.unique(y_pred_ternary, return_counts=True))
print(np.unique(y_pred_us, return_counts=True))

(array([0, 1, 2, 3]), array([ 69, 200,  29, 202]))
(array([0, 1, 3]), array([ 72, 214, 214]))


In [24]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred_ternary, target_names=LABEL))
print(classification_report(y_true, y_pred_us, target_names=LABEL))

                                    precision    recall  f1-score   support

                         Supported       0.68      0.45      0.54       122
                           Refuted       0.76      0.72      0.74       305
               Not Enough Evidence       0.49      0.54      0.51        35
Conflicting Evidence/Cherrypicking       0.16      0.37      0.22        38

                          accuracy                           0.62       500
                         macro avg       0.52      0.52      0.50       500
                      weighted avg       0.67      0.62      0.64       500

                                    precision    recall  f1-score   support

                         Supported       0.60      0.45      0.52       122
                           Refuted       0.74      0.76      0.75       305
               Not Enough Evidence       1.00      0.03      0.06        35
Conflicting Evidence/Cherrypicking       0.16      0.39      0.22        38

       

#### original bert - new dev500

                                    precision    recall  f1-score   support

                         Supported       0.58      0.33      0.42       122
                           Refuted       0.76      0.50      0.60       305
               Not Enough Evidence       0.14      0.11      0.12        35
Conflicting Evidence/Cherrypicking       0.11      0.58      0.18        38

                          accuracy                           0.44       500
                         macro avg       0.40      0.38      0.33       500
                      weighted avg       0.62      0.44      0.49       500

                                    precision    recall  f1-score   support

                         Supported       0.57      0.34      0.42       122
                           Refuted       0.77      0.54      0.63       305
               Not Enough Evidence       0.00      0.00      0.00        35
Conflicting Evidence/Cherrypicking       0.10      0.58      0.17        38

                          accuracy                           0.45       500
                         macro avg       0.36      0.36      0.31       500
                      weighted avg       0.61      0.45      0.50       500


#### originalbert - standard prediction dev

                                    precision    recall  f1-score   support

                         Supported       0.51      0.39      0.44       122
                           Refuted       0.72      0.71      0.71       305
               Not Enough Evidence       0.00      0.00      0.00        35
Conflicting Evidence/Cherrypicking       0.09      0.24      0.13        38

                          accuracy                           0.55       500
                         macro avg       0.33      0.33      0.32       500
                      weighted avg       0.57      0.55      0.55       500

                                    precision    recall  f1-score   support

                         Supported       0.50      0.39      0.44       122
                           Refuted       0.72      0.72      0.72       305
               Not Enough Evidence       0.00      0.00      0.00        35
Conflicting Evidence/Cherrypicking       0.09      0.24      0.13        38

                          accuracy                           0.55       500
                         macro avg       0.33      0.34      0.32       500
                      weighted avg       0.57      0.55      0.55       500

##### original bert - original gold dev

                                    precision    recall  f1-score   support

                         Supported       0.68      0.45      0.54       122
                           Refuted       0.76      0.72      0.74       305
               Not Enough Evidence       0.49      0.54      0.51        35
Conflicting Evidence/Cherrypicking       0.16      0.37      0.22        38

                          accuracy                           0.62       500
                         macro avg       0.52      0.52      0.50       500
                      weighted avg       0.67      0.62      0.64       500

                                    precision    recall  f1-score   support

                         Supported       0.60      0.45      0.52       122
                           Refuted       0.74      0.76      0.75       305
               Not Enough Evidence       1.00      0.03      0.06        35
Conflicting Evidence/Cherrypicking       0.16      0.39      0.22        38

                          accuracy                           0.61       500
                         macro avg       0.63      0.41      0.39       500
                      weighted avg       0.68      0.61      0.61       500

## New models - 3 labels

In [8]:
#higher version of transformers needed
%pip install --upgrade transformers

Note: you may need to restart the kernel to use updated packages.


In [9]:
class SequenceClassificationDataLoaderNLI(pl.LightningDataModule):
    def __init__(self, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer

    def tokenize_strings(
        self,
        source_sentences,
        max_length=400,
        pad_to_max_length=False,
        return_tensors="pt",
    ):
        encoded_dict = self.tokenizer(
            source_sentences,
            max_length=max_length,
            padding="max_length" if pad_to_max_length else "longest",
            truncation=True,
            return_tensors=return_tensors,
        )

        input_ids = encoded_dict["input_ids"]
        attention_masks = encoded_dict["attention_mask"]

        return input_ids, attention_masks

    def quadruple_to_string(self, claim, question, answer):
        if question is None:
            return [
                claim.strip(), answer.strip()
            ]
        else:
            return [
                claim.strip(), question.strip() + " " + answer.strip()
            ]
        

In [19]:
#model_name = "cross-encoder/nli-deberta-v3-small"
#model_name = "cross-encoder/nli-deberta-v3-large"
#model_name = "sileod/deberta-v3-small-tasksource-nli"
#model_name = "models/averitec/nli/deberta-v3-large/checkpoint-687"
#model_name = "models/averitec/nli/nli-deberta-v3-large/checkpoint-1145"
model_name = "models/averitec/nli_qa/deberta-v3-large/checkpoint-723"

examples = examples_dev500
#examples = examples_orig
#examples = examples_gold

In [20]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

id2correct_id = {0: 1, 1: 0, 2: 2}

device = "cuda:0" if torch.cuda.is_available() else "cpu"


model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

dataLoader_nli = SequenceClassificationDataLoaderNLI(tokenizer)

model.eval()

predictions = []

example_supports = []

y_pred_ternary = []
y_pred_us = []

for example in tqdm.tqdm(examples):
    example_strings = []
    #TODO handle without question differently!
    for evidence in example["evidence"]:
        example_strings.append(
            dataLoader_nli.quadruple_to_string(
                example["claim"], evidence["question"], evidence["answer"]
            )
        )

    if (
        len(example_strings) == 0
    ):  # If we found no evidence e.g. because google returned 0 pages, just output NEI.
        example["label"] = "Not Enough Evidence"
        y_pred_ternary.append(2)
        y_pred_us.append(2)
        continue

    tokenized_strings, attention_mask = dataLoader_nli.tokenize_strings(example_strings)
    with torch.no_grad():
        logits = model(tokenized_strings.to(device), attention_mask=attention_mask.to(device)).logits

    example_support = torch.argmax(
        logits,
        axis=1,
    )

    #remap the labels in example support tensor using the id2correct_id
    example_support = torch.tensor([id2correct_id[i.item()] for i in example_support])

    example_supports.append(example_support)

    has_unanswerable = False
    has_true = False
    has_false = False

    for v in example_support:
        if v == 0:
            has_true = True
        if v == 1:
            has_false = True
        if v in (
            2,
            3,
        ):  # TODO another hack -- we cant have different labels for train and test so we do this
            has_unanswerable = True

    if has_unanswerable:
        answer = 2
    elif has_true and not has_false:
        answer = 0
    elif not has_true and has_false:
        answer = 1
    else:
        answer = 3

    y_pred_ternary.append(answer)

    if has_true and has_false:
        answer_us = 3
    elif has_true and not has_false:
        answer_us = 0
    elif not has_true and has_false:
        answer_us = 1
    else:
        answer_us = 2 #otherwise NEI


    y_pred_us.append(answer_us)
    
#stack the supports to a tensor
example_supports = torch.stack(example_supports)

100%|██████████| 500/500 [00:27<00:00, 18.20it/s]


RuntimeError: stack expects each tensor to be equal size, but got [9] at entry 0 and [5] at entry 34

In [12]:
np.unique(example_supports.detach().cpu().numpy(), return_counts=True)

(array([0, 1, 2]), array([397, 883, 220]))

In [21]:
print(np.unique(y_pred_ternary, return_counts=True))
print(np.unique(y_pred_us, return_counts=True))

(array([0, 1, 2, 3]), array([ 55, 196, 205,  44]))
(array([0, 1, 2, 3]), array([ 98, 262,   4, 136]))


debera_qa original

(array([0, 1, 2, 3]), array([ 75, 224, 142,  59]))
(array([0, 1, 2, 3]), array([120, 286,  17,  77]))

deberta qa gold
(array([0, 1, 2, 3]), array([ 85, 262, 114,  39]))
(array([0, 1, 2, 3]), array([108, 319,  15,  58])

deberat qa dev500

(array([0, 1, 2, 3]), array([ 55, 196, 205,  44]))
(array([0, 1, 2, 3]), array([ 98, 262,   4, 136]))

deberta dev500

(array([0, 1, 2, 3]), array([ 34, 246, 202,  18]))
(array([0, 1, 2, 3]), array([ 88, 321,   5,  86]))

deberta dev_top_3_rerank_qa.json

(array([0, 1, 2, 3]), array([ 49, 252, 153,  46]))
(array([0, 1, 2, 3]), array([105, 317,  15,  63]))

deberta nli gold

(array([0, 1, 2, 3]), array([ 61, 261, 121,  57]))
(array([0, 1, 2, 3]), array([ 86, 318,  21,  75]))

deberta gold

(array([0, 1, 2, 3]), array([ 60, 296, 118,  26]))
(array([0, 1, 2, 3]), array([ 88, 347,  25,  40]))

large

(array([0, 1, 2, 3]), array([  6,  13, 479,   2]))
(array([0, 1, 2, 3]), array([ 93, 146, 221,  40]))

small
(array([0, 1, 2, 3]), array([  4,   7, 485,   4]))
(array([0, 1, 2, 3]), array([ 82, 169, 220,  29]))

In [22]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred_ternary, target_names=LABEL))
print(classification_report(y_true, y_pred_us, target_names=LABEL))

                                    precision    recall  f1-score   support

                         Supported       0.85      0.39      0.53       122
                           Refuted       0.86      0.55      0.67       305
               Not Enough Evidence       0.11      0.66      0.19        35
Conflicting Evidence/Cherrypicking       0.11      0.13      0.12        38

                          accuracy                           0.49       500
                         macro avg       0.48      0.43      0.38       500
                      weighted avg       0.75      0.49      0.56       500

                                    precision    recall  f1-score   support

                         Supported       0.67      0.54      0.60       122
                           Refuted       0.82      0.70      0.76       305
               Not Enough Evidence       0.25      0.03      0.05        35
Conflicting Evidence/Cherrypicking       0.13      0.47      0.21        38

       

deberta qa gold

                                    precision    recall  f1-score   support

                         Supported       0.87      0.61      0.71       122
                           Refuted       0.84      0.72      0.78       305
               Not Enough Evidence       0.20      0.66      0.31        35
Conflicting Evidence/Cherrypicking       0.13      0.13      0.13        38

                          accuracy                           0.65       500
                         macro avg       0.51      0.53      0.48       500
                      weighted avg       0.75      0.65      0.68       500

                                    precision    recall  f1-score   support

                         Supported       0.77      0.68      0.72       122
                           Refuted       0.82      0.86      0.84       305
               Not Enough Evidence       0.53      0.23      0.32        35
Conflicting Evidence/Cherrypicking       0.17      0.26      0.21        38

                          accuracy                           0.73       500
                         macro avg       0.57      0.51      0.52       500
                      weighted avg       0.74      0.73      0.73       500

deberta qa orig


                                    precision    recall  f1-score   support

                         Supported       0.56      0.34      0.43       122
                           Refuted       0.79      0.58      0.67       305
               Not Enough Evidence       0.11      0.43      0.17        35
Conflicting Evidence/Cherrypicking       0.07      0.11      0.08        38

                          accuracy                           0.47       500
                         macro avg       0.38      0.36      0.34       500
                      weighted avg       0.63      0.47      0.53       500

                                    precision    recall  f1-score   support

                         Supported       0.50      0.49      0.50       122
                           Refuted       0.79      0.74      0.76       305
               Not Enough Evidence       0.24      0.11      0.15        35
Conflicting Evidence/Cherrypicking       0.06      0.13      0.09        38

                          accuracy                           0.59       500
                         macro avg       0.40      0.37      0.37       500
                      weighted avg       0.62      0.59      0.60       500

deberta dev500

                                    precision    recall  f1-score   support

                         Supported       0.79      0.22      0.35       122
                           Refuted       0.79      0.64      0.70       305
               Not Enough Evidence       0.11      0.66      0.19        35
Conflicting Evidence/Cherrypicking       0.06      0.03      0.04        38

                          accuracy                           0.49       500
                         macro avg       0.44      0.39      0.32       500
                      weighted avg       0.69      0.49      0.53       500

                                    precision    recall  f1-score   support

                         Supported       0.59      0.43      0.50       122
                           Refuted       0.76      0.80      0.78       305
               Not Enough Evidence       0.20      0.03      0.05        35
Conflicting Evidence/Cherrypicking       0.10      0.24      0.15        38

                          accuracy                           0.61       500
                         macro avg       0.41      0.37      0.37       500
                      weighted avg       0.63      0.61      0.61       500


deberta dev_top_3_rerank_qa.json


                                    precision    recall  f1-score   support

                         Supported       0.69      0.28      0.40       122
                           Refuted       0.79      0.65      0.71       305
               Not Enough Evidence       0.11      0.49      0.18        35
Conflicting Evidence/Cherrypicking       0.11      0.13      0.12        38

                          accuracy                           0.51       500
                         macro avg       0.43      0.39      0.35       500
                      weighted avg       0.67      0.51      0.55       500

                                    precision    recall  f1-score   support

                         Supported       0.56      0.48      0.52       122
                           Refuted       0.77      0.80      0.78       305
               Not Enough Evidence       0.13      0.06      0.08        35
Conflicting Evidence/Cherrypicking       0.13      0.21      0.16        38

                          accuracy                           0.62       500
                         macro avg       0.40      0.39      0.38       500
                      weighted avg       0.62      0.62      0.62       500


deberta gold


                                    precision    recall  f1-score   support

                         Supported       0.88      0.43      0.58       122
                           Refuted       0.80      0.78      0.79       305
               Not Enough Evidence       0.17      0.57      0.26        35
Conflicting Evidence/Cherrypicking       0.15      0.11      0.12        38

                          accuracy                           0.63       500
                         macro avg       0.50      0.47      0.44       500
                      weighted avg       0.73      0.63      0.65       500

                                    precision    recall  f1-score   support

                         Supported       0.73      0.52      0.61       122
                           Refuted       0.77      0.87      0.82       305
               Not Enough Evidence       0.24      0.17      0.20        35
Conflicting Evidence/Cherrypicking       0.15      0.16      0.15        38

                          accuracy                           0.68       500
                         macro avg       0.47      0.43      0.44       500
                      weighted avg       0.67      0.68      0.67       500

small
                                    precision    recall  f1-score   support

                         Supported       0.75      0.02      0.05       122
                           Refuted       0.71      0.02      0.03       305
               Not Enough Evidence       0.07      1.00      0.13        35
Conflicting Evidence/Cherrypicking       0.00      0.00      0.00        38

                          accuracy                           0.09       500
                         macro avg       0.38      0.26      0.05       500
                      weighted avg       0.62      0.09      0.04       500

                                    precision    recall  f1-score   support

                         Supported       0.18      0.12      0.15       122
                           Refuted       0.69      0.38      0.49       305
               Not Enough Evidence       0.07      0.46      0.13        35
Conflicting Evidence/Cherrypicking       0.03      0.03      0.03        38

                          accuracy                           0.30       500
                         macro avg       0.25      0.25      0.20       500
                      weighted avg       0.47      0.30      0.35       500

large
                                    precision    recall  f1-score   support

                         Supported       0.33      0.02      0.03       122
                           Refuted       0.69      0.03      0.06       305
               Not Enough Evidence       0.07      1.00      0.14        35
Conflicting Evidence/Cherrypicking       0.00      0.00      0.00        38

                          accuracy                           0.09       500
                         macro avg       0.27      0.26      0.06       500
                      weighted avg       0.51      0.09      0.05       500

                                    precision    recall  f1-score   support

                         Supported       0.27      0.20      0.23       122
                           Refuted       0.64      0.31      0.42       305
               Not Enough Evidence       0.08      0.49      0.13        35
Conflicting Evidence/Cherrypicking       0.10      0.11      0.10        38

                          accuracy                           0.28       500
                         macro avg       0.27      0.28      0.22       500
                      weighted avg       0.47      0.28      0.33       500

## New Models 4 labels

In [33]:
model_name = "models/averitec/nli_4concat/deberta-v3-large/checkpoint-576"

examples = examples_dev500
#examples = examples_orig
#examples = examples_gold

In [34]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

id2correct_id = {0: 1, 1: 0, 2: 2, 3:3}

device = "cuda:0" if torch.cuda.is_available() else "cpu"


model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

dataLoader_nli = SequenceClassificationDataLoaderNLI(tokenizer)

model.eval()

y_pred = []


for example in tqdm.tqdm(examples):
    ev_string = ""
    for evidence in example["evidence"]:
        ev_string += evidence["question"] + " " + evidence["answer"] + " "

    example_strings = [dataLoader_nli.quadruple_to_string(example["claim"], None, ev_string[:-1])]

    tokenized_strings, attention_mask = dataLoader_nli.tokenize_strings(example_strings, max_length=1024)
    with torch.no_grad():
        logits = model(tokenized_strings.to(device), attention_mask=attention_mask.to(device)).logits

    example_support = torch.argmax(
        logits,
        axis=1,
    )

    label = id2correct_id[example_support.item()]


    y_pred.append(label)
    
print(y_pred)
print(np.unique(y_pred, return_counts=True))

100%|██████████| 500/500 [00:20<00:00, 24.60it/s]

[1, 1, 1, 1, 1, 1, 3, 0, 1, 0, 0, 3, 1, 0, 0, 0, 1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 0, 3, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 3, 3, 3, 3, 1, 0, 1, 3, 1, 3, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 3, 3, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 3, 1, 0, 0, 3, 1, 1, 0, 2, 1, 1, 1, 1, 3, 3, 1, 3, 1, 3, 3, 3, 1, 1, 1, 1, 1, 1, 0, 1, 0, 3, 1, 1, 1, 1, 1, 1, 3, 1, 3, 0, 3, 3, 0, 0, 1, 1, 0, 0, 1, 0, 3, 1, 3, 1, 1, 0, 1, 1, 0, 3, 1, 3, 2, 1, 0, 1, 0, 1, 1, 0, 3, 1, 3, 0, 1, 1, 1, 1, 1, 1, 3, 1, 0, 3, 0, 0, 3, 0, 1, 1, 1, 1, 1, 1, 1, 3, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 1, 0, 0, 3, 0, 1, 0, 1, 1, 1, 0, 3, 3, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 3, 3, 3, 1, 0, 1, 3, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 0, 1, 3, 0, 1, 1, 0, 0, 1, 2, 0, 1, 1, 1, 1, 3, 0, 1, 2, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 2, 1, 1, 1, 0, 3, 3, 0, 3, 2, 2, 1, 0, 0, 1, 0, 1, 1, 

In [35]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, target_names=LABEL))

                                    precision    recall  f1-score   support

                         Supported       0.69      0.71      0.70       122
                           Refuted       0.87      0.78      0.82       305
               Not Enough Evidence       0.27      0.09      0.13        35
Conflicting Evidence/Cherrypicking       0.14      0.32      0.19        38

                          accuracy                           0.68       500
                         macro avg       0.49      0.47      0.46       500
                      weighted avg       0.73      0.68      0.70       500



dev500

                                    precision    recall  f1-score   support

                         Supported       0.68      0.71      0.70       122
                           Refuted       0.87      0.79      0.82       305
               Not Enough Evidence       0.27      0.09      0.13        35
Conflicting Evidence/Cherrypicking       0.15      0.34      0.21        38

                          accuracy                           0.69       500
                         macro avg       0.49      0.48      0.47       500
                      weighted avg       0.73      0.69      0.70       500

gold

                                    precision    recall  f1-score   support

                         Supported       0.87      0.84      0.85       122
                           Refuted       0.91      0.84      0.88       305
               Not Enough Evidence       0.66      0.60      0.63        35
Conflicting Evidence/Cherrypicking       0.32      0.58      0.42        38

                          accuracy                           0.81       500
                         macro avg       0.69      0.72      0.69       500
                      weighted avg       0.84      0.81      0.82       500

orig

                                    precision    recall  f1-score   support

                         Supported       0.47      0.55      0.51       122
                           Refuted       0.83      0.52      0.64       305
               Not Enough Evidence       0.10      0.09      0.09        35
Conflicting Evidence/Cherrypicking       0.09      0.32      0.14        38

                          accuracy                           0.48       500
                         macro avg       0.37      0.37      0.34       500
                      weighted avg       0.64      0.48      0.53       500